In [1]:
from datetime  import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



In [2]:
# axis = 0 is Row, axis = 1 is Column
vehicle_routes = pd.read_csv("data/GothaerPlatz VehicleRoutes/Erfurt, Gothaer Platz_20211109_vehicle routes.csv")
len(vehicle_routes)


32703

In [3]:
# View the data in csv file for short
vehicle_routes.head()

,timestamp,origin,destination,class
0,2021-11-09 23:01:45.288000+00:00,B,D,CAR
1,2021-11-09 23:02:08.494000+00:00,D,B,CAR
2,2021-11-09 23:02:16.429000+00:00,D,B,CAR
3,2021-11-09 23:03:21.699000+00:00,D,B,CAR
4,2021-11-09 23:03:53.278000+00:00,D,A,CAR


In [4]:
##See the type of attributes
vehicle_routes.dtypes



timestamp      object
origin         object
destination    object
class          object
dtype: object

In [5]:
#some filtering
vehicle_routes[(vehicle_routes["class"] == "CAR") & (vehicle_routes["origin"] =="B") ]

,timestamp,origin,destination,class
0,2021-11-09 23:01:45.288000+00:00,B,D,CAR
6,2021-11-09 23:06:17.372000+00:00,B,D,CAR
7,2021-11-09 23:06:46.021000+00:00,B,A,CAR
10,2021-11-09 23:09:19.758000+00:00,B,D,CAR
16,2021-11-09 23:13:39.690000+00:00,B,A,CAR
...,...,...,...,...
32677,2021-11-10 22:41:22.361000+00:00,B,D,CAR
32685,2021-11-10 22:45:52.770000+00:00,B,D,CAR
32696,2021-11-10 22:56:30.760000+00:00,B,D,CAR
32699,2021-11-10 22:58:00.907000+00:00,B,D,CAR


In [6]:
# show all the cars with their frequencies and sort them
types_of_cars = vehicle_routes["class"].value_counts(sort=True)
types_of_cars





CAR                      27624
TRANSPORTER_WO_WINDOW     1703
TRANSPORTER_W_WINDOW      1248
TRUCK                      999
BICYCLE                    757
MOTORBIKE                  106
CAR_W_TRAILER              102
SEMI_TRAILER_TRUCK          68
TRUCK_W_TRAILER             51
BUS                         45
Name: class, dtype: int64

In [7]:
#change the the type of timestamp from simple string to datetime64[ns], which is in numpy
vehicle_routes['timestamp'] = pd.to_datetime(vehicle_routes['timestamp'], format="%Y-%m-%d %H:%M:%S.%f+00:00")



In [8]:
# Filter data in csv file and get the data on Nov 10 between 16 pM to 18 pM
all_vehicles_from_sixteen_to_eighteen = vehicle_routes.loc[(vehicle_routes['timestamp'] >= '2021-11-10 16:00:00.000000')
                                                           & (vehicle_routes['timestamp'] < '2021-11-10 18:00:00.000000')]



In [9]:
# get the data on Nov 10 between 16 pM to 16:15 pM
all_vehicles_from_sixteen_to_sixteenfifteen = all_vehicles_from_sixteen_to_eighteen.loc[(all_vehicles_from_sixteen_to_eighteen['timestamp'] >= '2021-11-10 07:00:00.000000')
                     & (all_vehicles_from_sixteen_to_eighteen['timestamp'] < '2021-11-10 07:15:00.000000')]



In [14]:
all_vehicles_from_sixteen_to_eighteen["class"].value_counts(sort=True)

CAR                      3575
TRANSPORTER_WO_WINDOW     139
TRANSPORTER_W_WINDOW       79
BICYCLE                    61
TRUCK                      34
MOTORBIKE                  21
BUS                         5
CAR_W_TRAILER               4
TRUCK_W_TRAILER             2
SEMI_TRAILER_TRUCK          2
Name: class, dtype: int64

In [11]:
#classify all vehicles from 16:00 to 18:00 according to the vehicle type and origin
classified_vehicles_from_sixteen_to_eighteen = pd.crosstab(all_vehicles_from_sixteen_to_eighteen["origin"],all_vehicles_from_sixteen_to_eighteen["class"])
classified_vehicles_from_sixteen_to_eighteen

class,BICYCLE,BUS,CAR,CAR_W_TRAILER,MOTORBIKE,SEMI_TRAILER_TRUCK,TRANSPORTER_WO_WINDOW,TRANSPORTER_W_WINDOW,TRUCK,TRUCK_W_TRAILER
origin,,,,,,,,,,
A,4,0,732,1,1,0,26,13,8,0
B,34,2,1141,2,1,1,58,16,14,0
D,23,3,1702,1,19,1,55,50,12,2


In [12]:
# consider all different types of trucks as one truck, all types of cars as one car, and all transporter as one transporter
# a_dataframe["AB"] = a_dataframe["A"] + a_dataframe["B"]
classified_vehicles_from_sixteen_to_eighteen["All_Trucks"] = classified_vehicles_from_sixteen_to_eighteen["TRUCK"] + classified_vehicles_from_sixteen_to_eighteen["TRUCK_W_TRAILER"]
classified_vehicles_from_sixteen_to_eighteen["All_Cars"] = classified_vehicles_from_sixteen_to_eighteen["CAR_W_TRAILER"] + classified_vehicles_from_sixteen_to_eighteen["CAR"]
classified_vehicles_from_sixteen_to_eighteen["All_Transporters"] = classified_vehicles_from_sixteen_to_eighteen["TRANSPORTER_WO_WINDOW"] + classified_vehicles_from_sixteen_to_eighteen["TRANSPORTER_W_WINDOW"]
classified_vehicles_from_sixteen_to_eighteen

class,BICYCLE,BUS,CAR,CAR_W_TRAILER,MOTORBIKE,SEMI_TRAILER_TRUCK,TRANSPORTER_WO_WINDOW,TRANSPORTER_W_WINDOW,TRUCK,TRUCK_W_TRAILER,All_Trucks,All_Cars,All_Transporters
origin,,,,,,,,,,,,,
A,4,0,732,1,1,0,26,13,8,0,8,733,39
B,34,2,1141,2,1,1,58,16,14,0,14,1143,74
D,23,3,1702,1,19,1,55,50,12,2,14,1703,105


In [13]:
#Drop some columns
summarized_vehicles = classified_vehicles_from_sixteen_to_eighteen.drop(columns=["BICYCLE","CAR","CAR_W_TRAILER","MOTORBIKE","SEMI_TRAILER_TRUCK","TRANSPORTER_WO_WINDOW","TRANSPORTER_W_WINDOW","TRUCK","TRUCK_W_TRAILER"])
summarized_vehicles


class,BUS,All_Trucks,All_Cars,All_Transporters
origin,,,,
A,0,8,733,39
B,2,14,1143,74
D,3,14,1703,105
